In [ ]:
import json
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    String,
    Text,
    Table,
    ForeignKey,
)
from sqlalchemy.orm import declarative_base, relationship, Session

# --- 1. DEFINE THE DATABASE STRUCTURE (The "Blueprint") ---

# The Base class is a standard starting point for all your data models.
Base = declarative_base()

# Many-to-many association table for Practices <--> SDGs
practice_sdg_association = Table(
    "practice_sdg_association",
    Base.metadata,
    Column("practice_id", String, ForeignKey("practices.id"), primary_key=True),
    Column("sdg_id", String, ForeignKey("sdgs.id"), primary_key=True),
)

# Many-to-many association table for Practices <--> Stakeholders
practice_stakeholder_association = Table(
    "practice_stakeholder_association",
    Base.metadata,
    Column("practice_id", String, ForeignKey("practices.id"), primary_key=True),
    Column("stakeholder_id", String, ForeignKey("stakeholders.id"), primary_key=True),
)


# Define the Practice "model" as a Python class
class Practice(Base):
    __tablename__ = "practices"
    id = Column(String, primary_key=True)
    name = Column(String, nullable=False)
    category = Column(String)
    capital_intensity = Column(Integer)
    technical_complexity = Column(Integer)
    description = Column(Text)

    # Define the relationships. SQLAlchemy will use the association tables
    # to manage these links automatically.
    sdgs = relationship(
        "SDG", 
        secondary=practice_sdg_association, 
        back_populates="practices"
    )
    stakeholders = relationship(
        "Stakeholder",
        secondary=practice_stakeholder_association,
        back_populates="practices",
    )

    def __repr__(self):
        return f"<Practice(name='{self.name}')>"


# Define the SDG model
class SDG(Base):
    __tablename__ = "sdgs"
    id = Column(String, primary_key=True)
    name = Column(String, nullable=False)
    goal = Column(Integer)
    description = Column(Text)

    practices = relationship(
        "Practice", 
        secondary=practice_sdg_association, 
        back_populates="sdgs"
    )

    def __repr__(self):
        return f"<SDG(name='{self.name}')>"


# Define the Stakeholder model
class Stakeholder(Base):
    __tablename__ = "stakeholders"
    id = Column(String, primary_key=True)
    name = Column(String, nullable=False)
    category = Column(String)

    practices = relationship(
        "Practice",
        secondary=practice_stakeholder_association,
        back_populates="stakeholders",
    )

    def __repr__(self):
        return f"<Stakeholder(name='{self.name}')>"


# --- 2. POPULATE THE DATABASE (The "Manufacturing Process") ---

def populate_database(session, data):
    """
    Populates the database with the initial dataset.
    This function is idempotent: it won't create duplicates if run again.
    """
    # Create dictionaries to easily look up the created objects by their ID
    practices_map = {}
    sdgs_map = {}
    stakeholders_map = {}

    # Create and add the main entities
    for p_data in data["practices"]:
        if not session.get(Practice, p_data["id"]):
            practice = Practice(**p_data)
            session.add(practice)
            practices_map[practice.id] = practice
    
    for s_data in data["sdgs"]:
        if not session.get(SDG, s_data["id"]):
            sdg = SDG(**s_data)
            session.add(sdg)
            sdgs_map[sdg.id] = sdg

    for sh_data in data["stakeholders"]:
        if not session.get(Stakeholder, sh_data["id"]):
            stakeholder = Stakeholder(**sh_data)
            session.add(stakeholder)
            stakeholders_map[stakeholder.id] = stakeholder
    
    # Commit to save the main entities so we can build relationships
    session.commit()

    # Now, build the relationships
    for rel in data["relationships"]:
        source_id = rel["source"]
        target_id = rel["target"]

        # Get the objects from the database
        practice = session.get(Practice, source_id)
        
        if practice:
            sdg = session.get(SDG, target_id)
            if sdg:
                if sdg not in practice.sdgs:
                    practice.sdgs.append(sdg)
                continue # Move to next relationship

            stakeholder = session.get(Stakeholder, target_id)
            if stakeholder:
                if stakeholder not in practice.stakeholders:
                    practice.stakeholders.append(stakeholder)

    # Commit the relationships
    session.commit()
    print("Database populated successfully.")





<Mine(name='Cerro Verde', country='Peru')>

In [2]:
# The user's data object
DATA = {
    "practices": [
        {"id": "p1", "name": "GHG Emissions Management", "category": "Environmental", "capital_intensity": 8, "technical_complexity": 9, "description": "Implementing technologies and processes to monitor, report, and reduce greenhouse gas emissions to combat climate change."},
        {"id": "p2", "name": "Water Management", "category": "Environmental", "capital_intensity": 7, "technical_complexity": 8, "description": "Comprehensive management of water use, recycling, and discharge to protect local water resources and ensure community access."},
        {"id": "p3", "name": "Overall Environmental Management", "category": "Environmental", "capital_intensity": 7, "technical_complexity": 7, "description": "Holistic practices to manage environmental impact, including biodiversity protection, waste management, and land rehabilitation."},
        {"id": "p4", "name": "Local Community Engagement", "category": "Social", "capital_intensity": 4, "technical_complexity": 5, "description": "Engaging with local communities to ensure benefits are shared, rights are respected, and economic opportunities are created."},
    ],
    "sdgs": [
        {"id": "sdg1", "name": "SDG 1: No Poverty", "goal": 1, "description": "End poverty in all its forms everywhere."},
        {"id": "sdg6", "name": "SDG 6: Clean Water and Sanitation", "goal": 6, "description": "Ensure availability and sustainable management of water and sanitation for all."},
        {"id": "sdg8", "name": "SDG 8: Decent Work and Economic Growth", "goal": 8, "description": "Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all."},
        {"id": "sdg13", "name": "SDG 13: Climate Action", "goal": 13, "description": "Take urgent action to combat climate change and its impacts."},
        {"id": "sdg15", "name": "SDG 15: Life on Land", "goal": 15, "description": "Protect, restore and promote sustainable use of terrestrial ecosystems."},
    ],
    "stakeholders": [
        {"id": "sh1", "name": "Local Communities", "category": "Civil Society"},
        {"id": "sh2", "name": "NGOs & CSOs", "category": "Civil Society"},
        {"id": "sh3", "name": "Media", "category": "Civil Society"},
        {"id": "sh4", "name": "Employees", "category": "Project/Internal"},
        {"id": "sh5", "name": "Trade Unions", "category": "Project/Internal"},
        {"id": "sh6", "name": "Suppliers", "category": "Project/Internal"},
        {"id": "sh7", "name": "Investors", "category": "Market & Financial"},
        {"id": "sh8", "name": "Shareholders", "category": "Market & Financial"},
        {"id": "sh9", "name": "Insurers", "category": "Market & Financial"},
        {"id": "sh10", "name": "Governments", "category": "Government & Regulatory"},
        {"id": "sh11", "name": "Local Authorities", "category": "Government & Regulatory"},
        {"id": "sh12", "name": "Industry Associations", "category": "Government & Regulatory"},
    ],
    "relationships": [
        {"source": "p1", "target": "sdg13"},
        {"source": "p2", "target": "sdg6"},
        {"source": "p3", "target": "sdg15"},
        {"source": "p4", "target": "sdg1"},
        {"source": "p4", "target": "sdg8"},
        {"source": "p1", "target": "sh2"},
        {"source": "p1", "target": "sh7"},
        {"source": "p1", "target": "sh10"},
        {"source": "p2", "target": "sh1"},
        {"source": "p2", "target": "sh11"},
        {"source": "p3", "target": "sh1"},
        {"source": "p3", "target": "sh2"},
        {"source": "p3", "target": "sh11"},
        {"source": "p4", "target": "sh1"},
        {"source": "p4", "target": "sh4"},
        {"source": "p4", "target": "sh5"},
    ]
}

# Create the database engine. This will create a file named 'mining_knowledge.db'
engine = create_engine("sqlite:///mining_knowledge.db")

# Create all the tables defined in our models
Base.metadata.create_all(engine)

# Create a session to interact with the database
with Session(engine) as session:
    populate_database(session, DATA)

    # --- QUERY EXAMPLE ---
    print("\n--- Query Example ---")
    # Get a specific practice and print its relationships
    ghg_practice = session.get(Practice, "p1")
    
    if ghg_practice:
        print(f"Practice: {ghg_practice.name}")
        
        print("  Related SDGs:")
        for sdg in ghg_practice.sdgs:
            print(f"    - {sdg.name}")

        print("  Related Stakeholders:")
        for stakeholder in ghg_practice.stakeholders:
            print(f"    - {stakeholder.name}")

Database populated successfully.

--- Query Example ---
Practice: GHG Emissions Management
  Related SDGs:
    - SDG 13: Climate Action
  Related Stakeholders:
    - Governments
    - NGOs & CSOs
    - Investors


In [4]:
import pandas as pd
from sqlalchemy import create_engine

# --- 1. Connect to the existing database ---
# This assumes the 'mining_knowledge.db' file is in the same folder.
engine = create_engine("sqlite:///mining_knowledge.db")






In [5]:
print("--- Reading the 'practices' table into a spreadsheet-like view ---")
# --- 2. Read a specific table into a Pandas DataFrame ---
# 'pd.read_sql_table' takes the table name and the connection engine.
practices_df = pd.read_sql_table("practices", engine)

--- Reading the 'practices' table into a spreadsheet-like view ---


In [6]:
practices_df

,id,name,category,capital_intensity,technical_complexity,description
0,p1,GHG Emissions Management,Environmental,8,9,Implementing technologies and processes to mon...
1,p2,Water Management,Environmental,7,8,"Comprehensive management of water use, recycli..."
2,p3,Overall Environmental Management,Environmental,7,7,Holistic practices to manage environmental imp...
3,p4,Local Community Engagement,Social,4,5,Engaging with local communities to ensure bene...


In [8]:

print("\n\n--- Reading the 'practice_stakeholder_association' table ---")
# You can also view the relationship tables to see the connections.
relationships_df = pd.read_sql_table("practice_stakeholder_association", engine)
print(relationships_df)



--- Reading the 'practice_stakeholder_association' table ---
   practice_id stakeholder_id
0           p1            sh2
1           p1            sh7
2           p1           sh10
3           p2            sh1
4           p2           sh11
5           p3            sh1
6           p3            sh2
7           p3           sh11
8           p4            sh1
9           p4            sh4
10          p4            sh5


In [9]:
stakeholders_df = pd.read_sql_table("stakeholders", engine)

In [10]:
stakeholders_df

,id,name,category
0,sh1,Local Communities,Civil Society
1,sh2,NGOs & CSOs,Civil Society
2,sh3,Media,Civil Society
3,sh4,Employees,Project/Internal
4,sh5,Trade Unions,Project/Internal
5,sh6,Suppliers,Project/Internal
6,sh7,Investors,Market & Financial
7,sh8,Shareholders,Market & Financial
8,sh9,Insurers,Market & Financial
9,sh10,Governments,Government & Regulatory


In [12]:
# Get the practice with id 'p1'
ghg_practice = session.get(Practice, "p1")
print(ghg_practice.name)
# Output: GHG Emissions Management

print("Stakeholders related to GHG Management:")
for stakeholder in ghg_practice.stakeholders:
    print(f"- {stakeholder.name} ({stakeholder.category})")

# Output:
# Stakeholders related to GHG Management:
# - NGOs & CSOs (Civil Society)
# - Investors (Market & Financial)
# - Governments (Government & Regulatory)

GHG Emissions Management
Stakeholders related to GHG Management:
- Governments (Government & Regulatory)
- NGOs & CSOs (Civil Society)
- Investors (Market & Financial)


In [ ]:
from sqlalchemy import select

# Create a query that joins Practice and Stakeholder
# and filters where the stakeholder's category is 'Civil Society'
stmt = ( # stmt is a common name for SQLAlchemy statements
    select(Practice)
    .join(Practice.stakeholders)
    .where(Stakeholder.category == "Civil Society")
    .distinct() # Use distinct to avoid duplicate practices
)

# Execute the query and get the results
civil_society_practices = session.scalars(stmt).all()

for p in civil_society_practices:
    print(p.name)

# Expected Output:
# GHG Emissions Management
# Overall Environmental Management
# Local Community Engagement

GHG Emissions Management
Water Management
Overall Environmental Management
Local Community Engagement
